In [1]:
# Importing libraries:

import pandas as pd
import numpy as np
import re
import gensim
import gensim.downloader as api
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import nltk

import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

import nltk

nltk.download('punkt', quiet=True)

from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords', quiet=True)

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Loading the data: 

BASE_PATH = "./dataset/"
# load dataset
df_metadata = pd.read_csv(BASE_PATH + "movies_metadata.csv")
# filter columns
metadata_cols = ["id", "title", "overview", "tagline", "genres", "original_language", "poster_path"]
df_metadata = df_metadata[metadata_cols]
# filter language
in_english = df_metadata["original_language"] == "en"
df_metadata_en = df_metadata[in_english]
df_metadata_en = df_metadata_en[[col for col in metadata_cols if col != "original_language"]]

print(len(df_metadata_en["overview"]))


overviews = []
for row in df_metadata_en["overview"]:
    overviews.append(str(row))

#print(len(overviews))

titles = []
for row in df_metadata_en["title"]:
    titles.append(str(row))


32269


/opt/software/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# Text cleaning / tokenizing finctions:

def remove_non_ascii(textstring):
    """This function removes non-ASCII characters from the text string"""
    return "".join(i for i in textstring if ord(i) < 128)


def tokenize_and_lowercase(textstring):
    """This function tokenizes the text string into lowercased tokens with Regex Tokenizer"""
    tokens = []
    tokenized_into_sentences = sent_tokenize(textstring)
    for e in tokenized_into_sentences:
        regex_tokenizer = re.compile("\w+[-'.]\w+|\w+")  # tokenizer removes whitespaces and punctuation
        tokenized = re.findall(regex_tokenizer, e)
        for e in tokenized:
            tokens.append(e.lower())
    return tokens


def remove_stop_words(tokenized_text, stop_words):
    """This function removes stop words from lowercased tokenized text"""
    clean_text = [x for x in tokenized_text if x not in stop_words]
    return clean_text


In [4]:
# Cleaning the movie overviews:

stop_words = set(stopwords.words("english"))  # setting the list of stopwords to get rid of
clean_overviews = []

for overview in overviews:
    overview_ascii = remove_non_ascii(overview)
    overview_tokenized_and_lowercased = tokenize_and_lowercase(overview_ascii)
    overview_clean = remove_stop_words(overview_tokenized_and_lowercased, stop_words)
    clean_overviews.append(overview_clean)



In [5]:
# Examples of output:

print(len(clean_overviews))

32269


In [6]:
# Making a corpus from overviews:

corpus = clean_overviews
print(corpus[9])
print(len(corpus))


['james', 'bond', 'must', 'unmask', 'mysterious', 'head', 'janus', 'syndicate', 'prevent', 'leader', 'utilizing', 'goldeneye', 'weapons', 'system', 'inflict', 'devastating', 'revenge', 'britain']
32269


In [7]:
# Loading pretrained word embedding from GoogleNews:

embeddings = api.load('word2vec-google-news-300', return_path=True)
GoogleNews_word2vec = KeyedVectors.load_word2vec_format(embeddings, binary=True)

[--------------------------------------------------] 1.4% 23.4/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=-------------------------------------------------] 3.8% 63.4/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[===-----------------------------------------------] 6.0% 99.8/1662.8MB downloaded

...

[========================================----------] 80.9% 1345.3/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[===============================================---] 94.9% 1578.7/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [8]:
# Training our corpus with GoogleNews embedding (CBOW architecture)
# [source for this piece: https://www.kdnuggets.com/2020/08/content-based-recommendation-system-word-embeddings.html]:

GoogleNews_model = Word2Vec(size=300, window=5, min_count=2, workers=-1, sg=0) # sg=0 indicates CBOW architecture
GoogleNews_model.build_vocab(corpus)
GoogleNews_model.intersect_word2vec_format(embeddings, lockf=1.0, binary=True)
GoogleNews_model.train(corpus, total_examples=GoogleNews_model.corpus_count, epochs=5)

(0, 0)

In [9]:
# Generating Word2Vec embeddings for each overview (CBOW architecture)
# [source for this piece: https://www.kdnuggets.com/2020/08/content-based-recommendation-system-word-embeddings.html]:

def vectors(clean_overviews):
    global word_embeddings
    word_embeddings = []
    
    for overview in clean_overviews:
        word2vec = None
        count = 0
        for word in overview:
          #  print(word)
            if word in GoogleNews_model.wv.vocab:
                count += 1
                if word2vec is None:
                    word2vec = GoogleNews_model[word]
                else:
                    word2vec = word2vec + GoogleNews_model[word]
                
        if word2vec is not None:
            word2vec = word2vec / count
        
        word_embeddings.append(word2vec)
    
    return word_embeddings

In [10]:
#print(len(vectors(clean_overviews)))
#print(type(vectors(clean_overviews)))
#embeddings=vectors(clean_overviews)
#print(np.array(embeddings[9]).reshape(1, -1))

#length = len(embeddings)
#print(length-1)
#print(np.array(embeddings[length-1]))
#idx=df_metadata.index[df_metadata['title'] == title].tolist()
#print(idx)
#print(np.array(embeddings[idx[0]]).reshape(1, -1))


In [11]:
# Function for recommending the top 5 similar movies (CBOW architecture):

def recommending(title):
    vectors_list=vectors(clean_overviews)
    idx=titles.index(title)
   # print(idx)
    length = len(vectors_list)
    similarities = []
    for i in range(0,length-1):
        try:
            similarity = cosine_similarity(np.array(vectors_list[idx]).reshape(1, -1), 
                                           np.array(vectors_list[i]).reshape(1, -1))
            similarities.append(similarity[0][0])
        except ValueError:
            similarities.append(0)
        
    similarities_dict={}
    for count, value in enumerate(similarities):
        similarities_dict[count]=value
        
    sorted_similarities_dict = dict(sorted(similarities_dict.items(),
                           key=lambda item: item[1],
                           reverse=True))
    
    indices_top=list(sorted_similarities_dict.keys())[1:6]
    print("Top 5 movies most similar to", title)
    print("===================================")
    for index in indices_top:
        print(titles[index])
        print("\n")
        print(overviews[index])
        print("\n")

In [12]:
# Testing the recommendations (CBOW architecture):

title="GoldenEye"
recommending(title)

title="Deathline"
recommending(title)

/tmp/ipykernel_137/3366340677.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  word2vec = GoogleNews_model[word]
/tmp/ipykernel_137/3366340677.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  word2vec = word2vec + GoogleNews_model[word]


Top 5 movies most similar to GoldenEye
From Russia with Love


Agent 007 is back in the second installment of the James Bond series, this time battling a secret crime organization known as SPECTRE. Russians Rosa Klebb and Kronsteen are out to snatch a decoding device known as the Lektor, using the ravishing Tatiana to lure Bond into helping them. Bond willingly travels to meet Tatiana in Istanbul, where he must rely on his wits to escape with his life in a series of deadly encounters with the enemy


Johnny English Reborn


The most prominent heads of state in the world begin gathering for a conference that could have a major impact on global politics. When MI-7 receives word that the Chinese premier has become the target of some high-powered killers, it falls on Johnny English to save the day. Armed with the latest high-tech weaponry and gadgets that would make even James Bond jealous, the once-disgraced agent uncovers evidence of a massive conspiracy involving some of the world's mos

/tmp/ipykernel_137/3366340677.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  word2vec = GoogleNews_model[word]
/tmp/ipykernel_137/3366340677.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  word2vec = word2vec + GoogleNews_model[word]


Top 5 movies most similar to Deathline
Ninja: Shadow of a Tear


Fight everyone and trust no one: it's the code of survival practiced by martial-arts master Casey Bowman after his life of domestic bliss is shattered by a savage act of violence. Vowing revenge, the fearless American stealthily tracks the killer from Osaka to Bangkok to Rangoon with the help of a wise and crafty sensei. His only clues: a series of victims whose necks bear the distinctive mark of strangulation by barbed wire. Fighting to avenge as well as to survive, Casey must sharpen his razor-like responses and take his battle skills to the next level, even using deep meditation to fake his own death. His target: the sinister drug lord Goro, who is flooding the streets with deadly meth cooked at his remote jungle factory. To prepare for his ultimate confrontation, Casey must finally become an invisible warrior worthy of the name Ninja. But just when his prey is cornered, an unexpected twist shows Casey that his battle 

In [13]:
# Training our corpus with GoogleNews embedding (Skip-Gram architecture)
# [source for this piece: https://www.kdnuggets.com/2020/08/content-based-recommendation-system-word-embeddings.html]:

GoogleNews_model_SG = Word2Vec(size=300, window=5, min_count=2, workers=-1, sg=1) # sg=1 indicates Skip-Gram architecture
GoogleNews_model_SG.build_vocab(corpus)
GoogleNews_model_SG.intersect_word2vec_format(embeddings, lockf=1.0, binary=True)
GoogleNews_model_SG.train(corpus, total_examples=GoogleNews_model_SG.corpus_count, epochs=5)

(0, 0)

In [14]:
# Generating Word2Vec embeddings for each overview (Skip-Gram architecture)
# [source for this piece: https://www.kdnuggets.com/2020/08/content-based-recommendation-system-word-embeddings.html]:

def vectors_SG(clean_overviews):
    global word_embeddings_SG
    word_embeddings_SG = []
    
    for overview in clean_overviews:
        word2vec = None
        count = 0
        for word in overview:
          #  print(word)
            if word in GoogleNews_model_SG.wv.vocab:
                count += 1
                if word2vec is None:
                    word2vec = GoogleNews_model_SG[word]
                else:
                    word2vec = word2vec + GoogleNews_model_SG[word]
                
        if word2vec is not None:
            word2vec = word2vec / count
        
        word_embeddings_SG.append(word2vec)
    
    return word_embeddings_SG

In [15]:
# Function for recommending the top 5 similar movies (Skip-Gram architecture):

def recommending(title):
    vectors_list=vectors_SG(clean_overviews)
    idx=titles.index(title)
   # print(idx)
    length = len(vectors_list)
    similarities = []
    for i in range(0,length-1):
        try:
            similarity = cosine_similarity(np.array(vectors_list[idx]).reshape(1, -1), 
                                           np.array(vectors_list[i]).reshape(1, -1))
            similarities.append(similarity[0][0])
        except ValueError:
            similarities.append(0)
        
    similarities_dict={}
    for count, value in enumerate(similarities):
        similarities_dict[count]=value
        
    sorted_similarities_dict = dict(sorted(similarities_dict.items(),
                           key=lambda item: item[1],
                           reverse=True))
    
    indices_top=list(sorted_similarities_dict.keys())[1:6]
    print("Top 5 movies most similar to", title)
    print("===================================")
    for index in indices_top:
        print(titles[index])
        print("\n")
        print(overviews[index])
        print("\n")

In [17]:
# Testing the recommendations (Skip-Gram architecture):

title="Deathline"
recommending(title)


# https://image.tmdb.org/t/p/original/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg

/tmp/ipykernel_137/2790036778.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  word2vec = GoogleNews_model_SG[word]
/tmp/ipykernel_137/2790036778.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  word2vec = word2vec + GoogleNews_model_SG[word]


Top 5 movies most similar to Deathline
Ninja: Shadow of a Tear


Fight everyone and trust no one: it's the code of survival practiced by martial-arts master Casey Bowman after his life of domestic bliss is shattered by a savage act of violence. Vowing revenge, the fearless American stealthily tracks the killer from Osaka to Bangkok to Rangoon with the help of a wise and crafty sensei. His only clues: a series of victims whose necks bear the distinctive mark of strangulation by barbed wire. Fighting to avenge as well as to survive, Casey must sharpen his razor-like responses and take his battle skills to the next level, even using deep meditation to fake his own death. His target: the sinister drug lord Goro, who is flooding the streets with deadly meth cooked at his remote jungle factory. To prepare for his ultimate confrontation, Casey must finally become an invisible warrior worthy of the name Ninja. But just when his prey is cornered, an unexpected twist shows Casey that his battle 